# 유튜브 랭킹 데이터 수집하기

In [ ]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
#.py형태로 돌릴경우 time.sleep(2)

In [ ]:
trs = driver.find_elements_by_css_selector('.aos-init')

In [ ]:
len(trs)

100

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
channel_list = soup.select('.aos-init')

In [ ]:
len(channel_list)

100

In [ ]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip()
category

'[음악/댄스/가수]'

In [ ]:
name = channel.select_one('.subject a').text.strip()

In [ ]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('6400만', '190억0381만', '371')

In [ ]:
channels = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]') #strip().strip() 두번써도됨
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]
    channels.append([category, name, subscriber, view, video]) 

In [ ]:
df = pd.DataFrame(channels, columns=['카테고리', '채널명', '구독자수', '조회수', '비디오수'])

In [ ]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,482
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6557만,433
97,음악/댄스/가수,NCT DREAM,386만,3억7422만,228
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,"1,847"
99,애완/반려동물,크림히어로즈,381만,13억7401만,939


# 분석을 위한 만과 억을 숫자로 바꿔주는 함수

In [71]:
def convert_unit(s):
  #  s = ' '.join(s.split('억')) #억을 분리하고 다시 붙이기
    s = s.replace('억', '').replace('개','').replace(',','')
    s = s.replace('만', '0000')
    return s
    return f'{int(s):,d}'

In [ ]:
convert_unit('1억122만')

'11220000'

- 두번째 페이지 -XPath로 찾아 click()하여 이동하기

In [ ]:
 driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()
 

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [ ]:
result = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]') #strip().strip() 두번써도됨
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]
    result.append([category, name, subscriber, view, video]) 

In [ ]:
df = pd.DataFrame(result, columns=['카테고리', '채널명', '구독자수', '조회수', '비디오수'])
df

,카테고리,채널명,구독자수,조회수,비디오수
0,BJ/인물/연예인,SUNBA선바,80만,9억5018만,"2,219"
1,TV/방송,fantagio 판타지오,80만,1억7504만,213
2,영화/만화/애니,신비아파트 공식채널 - Shinbi House,80만,4억4293만,"1,052"
3,패션/미용,areumsongee아름송이,80만,6617만,177
4,미분류,TOP MEDIA,80만,1억5960만,497
...,...,...,...,...,...
95,BJ/인물/연예인,외질혜 (OzilTube),71만,4억3384만,"1,587"
96,TV/방송,나혼자산다 STUDIO,71만,2억2243만,287
97,패션/미용,헤이즐 Heizle,71만,7445만,290
98,음악/댄스/가수,HEADLINER,71만,1억6786만,186


## page 1 ~ 10 까지 크롤링하기

In [ ]:
results = []

for page in range(1, 11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')
    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')  
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category, name, subscriber, view, video]) 

In [ ]:
df = pd.DataFrame(results, columns=['카테고리', '채널명', '구독자수', '조회수', '비디오수'])
df


,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,64000000,19003810000,371
1,음악/댄스/가수,HYBE LABELS,60300000,18713050000,654
2,음악/댄스/가수,BANGTANTV,56400000,12194960000,1579
3,음악/댄스/가수,SMTOWN,28500000,21870920000,3729
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],26500000,11052880000,223
...,...,...,...,...,...
995,게임,미소,510000,210770000,3084
996,미분류,밉지않은 관종언니,510000,67740000,170
997,미분류,복지마블TV [Welfare Marble],510000,49960000,240
998,음식/요리/레시피,Muggo,510000,183720000,1388


In [ ]:
df.to_csv('유튜브_순위.csv', index=False)
 

In [ ]:
driver.close()

# Selenium 만 써서 동일한 결과가 나오도록 